# 1. Libraries
---

In [1]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, DateTime, MetaData, ForeignKey, delete, text
from datetime import datetime

# access 
import os
from dotenv import load_dotenv
load_dotenv()

True

# 2. Extract
---

In [2]:
#import
data= pd.read_csv('../data/spam.csv', encoding="iso-8859-1")
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# 3. Transform
---

In [3]:
# current datetime
now = datetime.now()
ts = int(datetime.timestamp(now) * 1000)

In [4]:
data = data.drop_duplicates()
data['content'] = data[['v2','Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']].fillna('').agg(' '.join, axis=1) #NaN need to be replace by no value .fillna()
data.drop(columns=['v2','Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1, inplace=True)
data = data.rename(columns={"v1": "label"})
data.index.rename('id', inplace=True)
data['created_at'] = ts


In [5]:
data.head()

,label,content,created_at
id,,,
0,ham,"Go until jurong point, crazy.. Available only ...",1761603265172
1,ham,Ok lar... Joking wif u oni...,1761603265172
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1761603265172
3,ham,U dun say so early hor... U c already then say...,1761603265172
4,ham,"Nah I don't think he goes to usf, he lives aro...",1761603265172


# 4. Load
---

In [6]:
# Credentials
DATABASE = os.environ["POSTGRES_DATABASE"]

#start engine
engine = create_engine(DATABASE)

# Connect to the db
conn = engine.connect()
meta = MetaData()

# Table "messages_spam_detector"
messages_spam_detector = Table(
    "messages_spam_detector", meta,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("content", String),
    Column("label", String),
    Column("label_predict", String),
    Column("created_at", String)
)

# Create table
meta.create_all(engine)

# Delete if not empty
stmt_delete_message = delete(messages_spam_detector)


conn.execute(stmt_delete_message)
conn.commit()

In [7]:
data.to_sql("messages_spam_detector", engine, if_exists = "append")

169